<a href="https://colab.research.google.com/github/Swayamprakashpatel/DE_Interact.github.io/blob/main/DE_Interact.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**DRUG-EXCIPIENT INTERACTION PREDICTION TOOL**

**Please Cite this our work as:**

Swayamprakash Patel, Mehul Patel, Mangesh Kulkarni, Mruduka S. Patel,
DE-INTERACT: A machine-learning-based predictive tool for the drug-excipient interaction study during product development—Validation through Paracetamol and Vanillin as a case study, International Journal of Pharmaceutics, 2023, 122839, ISSN 0378-5173.

https://doi.org/10.1016/j.ijpharm.2023.122839

https://www.sciencedirect.com/science/article/pii/S0378517323002594

**Dr. Swayamprakash Patel**

Email: swayamprakashpatel.ph@charusat.ac.in

**Model Details:**

Training Accuracy is 0.9898828268051147

Validation Accuracy is 0.9527363181114197

Training loss is 0.020730653777718544

Validation loss is 0.18604408204555511

**FOR SINGLE PREDICTION**

In [ ]:
#@title Add PubChem Compound ID (CID) for both Drug and Excipient and Run Both Cells (Shift + Enter)
#%%capture
!pip install pubchempy
!pip install gdown

import tensorflow as tf
import pandas as pd
import numpy as np
import io
from tensorflow import keras
import pubchempy as pcp
from sklearn.preprocessing import OneHotEncoder
from keras.models import load_model
import time as tm
import gdown

file_id = '13BpkcW3IRhbVWAVHK7JPwtB7WCSyG73W'  # Replace with your actual file ID
destination = '/content/sample_data/Model.hdf5'  # Desired destination path

gdown.download(f'https://drive.google.com/uc?id={file_id}', destination, quiet=False)

#Load Model
model = load_model('/content/sample_data/Model.hdf5', compile= False)

#Compile Fingerprints (1762 Inputs)
Drug_CID =  5819#@param {type:"number"}
Excipient_CID =  6251#@param {type:"number"}

list(pcp.Compound.from_cid(Drug_CID).cactvs_fingerprint)
CID_D = pcp.Compound.from_cid(Drug_CID)
CID_E = pcp.Compound.from_cid(Excipient_CID)

FPD = CID_D.cactvs_fingerprint
FPE = CID_E.cactvs_fingerprint

List1 = list(FPD)
List2 = list(FPE)
List = List1 + List2

t = pd.DataFrame(np.array(List).reshape(-1,len(List)))

dataset1 = t.values
#t.dtype = int
X_Predict = (dataset1[:,0:1762].astype(int))
#print(X_Predict)
#print(X_Predict.shape)
#print(X_Predict.dtype)

t1 = model.predict(X_Predict)
#print(t1)

t2 = np.round(t1)
#print(t2)

#Y_prediction = enc.inverse_transform(t2)
#print(Y_prediction, Y_prediction.shape)
Y1 = np.array([['Possibally Compatible'], ['Possibally Incompatible']])
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(Y1)
OneHotEncoder(handle_unknown='ignore')
enc.categories_
Y = enc.transform(Y1).toarray()
Y_prediction = enc.inverse_transform(t2)

#print(Y_prediction)

# Define ANSI escape codes for colors
RED = '\033[1;37;41m'  # White text on red background
GREEN = '\033[1;37;42m'  # White text on green background
RESET = '\033[0m'  # Reset color

# Check the prediction and choose the appropriate color and probability
if Y_prediction == 'Possibally Incompatible':
    highlight = RED
    probability = t1[0][1]  # Second value for incompatibility
else:
    highlight = GREEN
    probability = t1[0][0]  # First value for compatibility

# Print the result with color and the selected probability
print(f"{highlight}{Y_prediction} (Probability is {probability}){RESET}")

**IF NO CID in PubChem, USE CANONICAL SMILE** **Use SMILE Generator utility of [Cheminfo](https://www.cheminfo.org) or any other tool.

In [ ]:
#@title Add SMILE for both Drug and Excipient and Run Both Cells (Shift + Enter)
#%%capture
!pip install pubchempy
!pip install padelpy
!pip install gdown

import tensorflow as tf
import pandas as pd
import numpy as np
import io
from tensorflow import keras
import pubchempy as pcp
from sklearn.preprocessing import OneHotEncoder
from keras.models import load_model
import time as tm
import gdown

file_id = '13BpkcW3IRhbVWAVHK7JPwtB7WCSyG73W'  # Replace with your actual file ID
destination = '/content/sample_data/Model.hdf5'  # Desired destination path

gdown.download(f'https://drive.google.com/uc?id={file_id}', destination, quiet=False)

#Load Model
model = load_model('/content/sample_data/Model.hdf5', compile= False)

#Compile Fingerprints (1762 Inputs)
Drug_SMILE = "C1CO1.C(C(CO)O)O" #@param {type:"string"}
Excipient_SMILE = "COC1=C(C=CC(=C1)C=O)O" #@param {type:"string"}

CID_D = from_smiles(Drug_SMILE, fingerprints=True, descriptors=False)
CID_E = from_smiles(Excipient_SMILE, fingerprints=True, descriptors=False)

FPD = list(CID_D.values())
FPE = list(CID_E.values())

List = FPD + FPE

t = pd.DataFrame(np.array(List).reshape(-1,len(List)))

t = pd.DataFrame(np.array(List).reshape(-1,len(List)))

dataset1 = t.values
#t.dtype = int
X_Predict = (dataset1[:,0:1762].astype(int))
#print(X_Predict)
#print(X_Predict.shape)
#print(X_Predict.dtype)

t1 = model.predict(X_Predict)
#print(t1)

t2 = np.round(t1)
#print(t2)

#Y_prediction = enc.inverse_transform(t2)
#print(Y_prediction, Y_prediction.shape)
Y1 = np.array([['Possibally Compatible'], ['Possibally Incompatible']])
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(Y1)
OneHotEncoder(handle_unknown='ignore')
enc.categories_
Y = enc.transform(Y1).toarray()
Y_prediction = enc.inverse_transform(t2)

#print(Y_prediction)

# Define ANSI escape codes for colors
RED = '\033[1;37;41m'  # White text on red background
GREEN = '\033[1;37;42m'  # White text on green background
RESET = '\033[0m'  # Reset color

## Check the prediction and choose the appropriate color and probability
if Y_prediction == 'Possibally Incompatible':
    highlight = RED
    probability = t1[0][1]  # Second value for incompatibility
else:
    highlight = GREEN
    probability = t1[0][0]  # First value for compatibility

# Print the result with color and the selected probability
print(f"{highlight}{Y_prediction} (Probability is {probability}){RESET}")

**FOR MULTIPLE PREDICTION: Download Predict.xlsx file, Add drug CID in each row and predict using below option**
[Download Predict.xlsx](https://github.com/Swayamprakashpatel/DE_Interact.github.io/blob/6e6a33a3f340b373114542ff15319686a6461636/Predict.xlsx)

In [ ]:
# @title Uplod Excel (Predict.xlsx)
# %%capture
!pip install pubchempy
!pip install gdown
!pip install openpyxl # Install this for Excel file support

import tensorflow as tf
import pandas as pd
import numpy as np
import io
from tensorflow import keras
import pubchempy as pcp
from sklearn.preprocessing import OneHotEncoder
from keras.models import load_model
import time
import gdown
from google.colab import files

# Define the file ID for the model and download it
file_id = '13BpkcW3IRhbVWAVHK7JPwtB7WCSyG73W'
destination = '/content/sample_data/Model.hdf5'
gdown.download(f'https://drive.google.com/uc?id={file_id}', destination, quiet=True)

# Load the pre-trained model
model = load_model(destination, compile=False)

# Define a function to get fingerprints and make a prediction
def predict_compatibility(drug_cid, excipient_cid):
    """
    Takes drug and excipient CIDs, gets their fingerprints,
    and predicts their compatibility using the loaded model.
    Includes a delay to prevent 'ServerBusy' errors.
    """
    try:
        # Add a delay before making the PubChem API calls to avoid 'ServerBusy' errors.
        time.sleep(1) # Pause for 1 second

        cid_d = pcp.Compound.from_cid(int(drug_cid))
        cid_e = pcp.Compound.from_cid(int(excipient_cid))

        fpd = cid_d.cactvs_fingerprint
        fpe = cid_e.cactvs_fingerprint

        list1 = list(fpd)
        list2 = list(fpe)
        input_list = list1 + list2

        if len(input_list) != 1762:
            return "Error: Fingerprint length mismatch"

        x_predict = np.array(input_list).reshape(1, -1).astype(int)
        prediction_raw = model.predict(x_predict, verbose=0)

        # Determine the final prediction and probability
        if prediction_raw[0][0] > prediction_raw[0][1]:
            result = "Possibly Compatible"
            probability = prediction_raw[0][0]
        else:
            result = "Possibly Incompatible"
            probability = prediction_raw[0][1]

        return f"{result} (Probability: {probability:.4f})"

    except Exception as e:
        return f"Prediction Error: {e}"

# --- User Interaction and File Processing ---

# Prompt user to upload their file
print("Please upload your CSV or Excel file with 'Drug_CID' and 'Excipient_CID' columns.")
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

# Load the data based on file type
if file_name.endswith('.csv'):
    df = pd.read_csv(io.StringIO(uploaded[file_name].decode('utf-8')))
elif file_name.endswith(('.xls', '.xlsx')):
    df = pd.read_excel(io.BytesIO(uploaded[file_name]))
else:
    print("Unsupported file format. Please upload a .csv, .xls, or .xlsx file.")
    exit()

# Add a new column for the prediction results
df['Compatibility_Prediction'] = ""

# Process each row and store the prediction
for index, row in df.iterrows():
    drug_cid = row['Drug_CID']
    excipient_cid = row['Excipient_CID']

    # Check if CIDs are valid numbers before processing
    if pd.isna(drug_cid) or pd.isna(excipient_cid):
        df.at[index, 'Compatibility_Prediction'] = "Invalid CID"
        continue

    # Make the prediction
    result = predict_compatibility(drug_cid, excipient_cid)
    df.at[index, 'Compatibility_Prediction'] = result
    print(f"Processed row {index + 1}/{len(df)}: {result}")

# Save the final DataFrame to a new file
output_file_name = f"Results_{file_name}"
if file_name.endswith('.csv'):
    df.to_csv(output_file_name, index=False)
elif file_name.endswith(('.xls', '.xlsx')):
    df.to_excel(output_file_name, index=False)

print(f"\nProcessing complete! The results have been saved to '{output_file_name}'.")
# Automatically download the generated file
files.download(output_file_name)